# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-33.48,100,88,1.05,RU,1707971185
1,1,keflavik,64.0049,-22.5624,1.24,93,40,4.12,IS,1707971185
2,2,ushuaia,-54.8000,-68.3000,7.81,66,0,9.26,AR,1707971185
3,3,talnakh,69.4865,88.3972,-25.66,94,87,1.68,RU,1707971186
4,4,west island,-12.1568,96.8225,30.99,58,86,5.66,CC,1707971186


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = 'Humidity',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.6,
    color = "City",
    alpha=0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp =  (city_data_df['Max Temp'] < 28) & (city_data_df['Max Temp'] > 17)
wind_speed = city_data_df['Wind Speed'] < 7
cloudiness = city_data_df['Cloudiness'] == 0

# Filter cities 
filtered_cities_df = city_data_df[max_temp & wind_speed & cloudiness]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,imperatriz,-5.5264,-47.4917,26.03,83,0,0.00,BR,1707971194
108,108,dwarka,22.2394,68.9678,22.91,32,0,6.51,IN,1707971200
122,122,pisco,-13.7000,-76.2167,24.03,78,0,4.63,PE,1707971202
182,182,callao,-12.0667,-77.1500,27.07,69,0,6.69,PE,1707970942
214,214,villa gesell,-37.2639,-56.9730,17.25,77,0,4.88,AR,1707970971
223,223,nsanje,-16.9200,35.2620,21.47,88,0,0.21,MW,1707971216
248,248,suratgarh,29.3167,73.9000,18.35,16,0,0.19,IN,1707971218
265,265,boa vista,2.8197,-60.6733,26.99,78,0,3.60,BR,1707971220
314,314,teguise,29.0605,-13.5640,18.12,77,0,3.60,ES,1707971227
378,378,al khaburah,23.9886,57.0984,22.13,79,0,1.56,OM,1707971237


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
63,imperatriz,BR,-5.5264,-47.4917,83,
108,dwarka,IN,22.2394,68.9678,32,
122,pisco,PE,-13.7000,-76.2167,78,
182,callao,PE,-12.0667,-77.1500,69,
214,villa gesell,AR,-37.2639,-56.9730,77,
223,nsanje,MW,-16.9200,35.2620,88,
248,suratgarh,IN,29.3167,73.9000,16,
265,boa vista,BR,2.8197,-60.6733,78,
314,teguise,ES,29.0605,-13.5640,77,
378,al khaburah,OM,23.9886,57.0984,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'api_key' : geoapify_key,
    'limit' : 1
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
imperatriz - nearest hotel: Hotel Posseidon
dwarka - nearest hotel: The Dwarika Hotel
pisco - nearest hotel: La Portada
callao - nearest hotel: Casa Ronald
villa gesell - nearest hotel: Hotel El Faro
nsanje - nearest hotel: No hotel found
suratgarh - nearest hotel: The Golden Inn
boa vista - nearest hotel: Aipana Plaza Hotel
teguise - nearest hotel: Finca Las Laderas
al khaburah - nearest hotel: السندس للشقق الفندقية
manta - nearest hotel: Hotel Santa Fe
gisborne - nearest hotel: Portside Hotel
kalur kot - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
63,imperatriz,BR,-5.5264,-47.4917,83,Hotel Posseidon
108,dwarka,IN,22.2394,68.9678,32,The Dwarika Hotel
122,pisco,PE,-13.7000,-76.2167,78,La Portada
182,callao,PE,-12.0667,-77.1500,69,Casa Ronald
214,villa gesell,AR,-37.2639,-56.9730,77,Hotel El Faro
223,nsanje,MW,-16.9200,35.2620,88,No hotel found
248,suratgarh,IN,29.3167,73.9000,16,The Golden Inn
265,boa vista,BR,2.8197,-60.6733,78,Aipana Plaza Hotel
314,teguise,ES,29.0605,-13.5640,77,Finca Las Laderas
378,al khaburah,OM,23.9886,57.0984,79,السندس للشقق الفندقية


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size = 'Humidity',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.6,
    color = "City",
    alpha=0.5,
    hover_cols=['Country','Hotel Name']
)

# Display the map
hotel_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)